<div style="max-width:1200px"><img src="../_resources/mgnify_banner.png" width="100%"></div>

<img src="../_resources/mgnify_logo.png" width="200px">

# Browsing the MGnify website

**Aims:**

- **Search for analyses using the MGnify website**
- **Understand hierarchy of data and metadata**
- **Understand the analysis types and outputs**
- **Access the MGnify API programmatically**
- **Explore the Genomes resource in MGnify**

<div style="background: #d0debb; padding: 16px; border-radius: 4px; margin: 8px">
This is an interactive code notebook (a Jupyter Notebook). To run this code, click into each cell and press the ▶ button in the top toolbar, or press shift+enter
</div>

## Section 1: explore the MGnify website and fetch data

This section will guide you through browsing MGnify studies and analyses and associated metadata. The tasks are a combination of clicking through the website and running code. Keep a note of your answers to discuss later. 

For this practical we will look at the fish gut metagenome. Specifically the [HoloFood](https://www.holofood.eu/) study. You can repeat these practicals anytime with a different biome of your choice.

**Task 1**: Go to the [MGnify](https://www.ebi.ac.uk/metagenomics) website home page. Using the **"Text Search"** feature, search for studies:
* belonging to the Host associated, fish digestive system

There are numerous Fish gut studies available. The samples were submitted for the HoloFood project by partners at the University of Copenhagen. 
* With this information, filter the studies further to obtain only the HoloFood studies
* Select the study for samples 'Trial A and B' 

<div style="background: #bbd2de; padding: 16px; border-radius: 4px; margin: 8px">
Hint: Make use of the filters on the left hand side
</div>

<div style="background: #d0debb; padding: 16px; border-radius: 4px; margin: 8px">
Fun extras! You can find the same study and related HoloFood project studies using the 'Browse Studies' --> 'Super studies' function
</div>

**Task 2**: Go to the [study overview page](https://www.ebi.ac.uk/metagenomics/studies/MGYS00006041#overview) page if you haven't yet.

<div style="background: #e8d1e8; padding: 16px; border-radius: 4px; margin: 8px">
Question: Off the coast of which country were all of the study's samples collected?
</div>

<div style="background: #e8d1e8; padding: 16px; border-radius: 4px; margin: 8px">
Question: How many analyses are in the study?
</div>

There is not (yet) a publication associated with this data. So we have to work a little harder to find some metadata. 

**Task 3**: Scroll down to the analyses and select any sample. There are links to more detailed metadata in BioSamples. Click on the link out to the BioSample. 

<div style="background: #e8d1e8; padding: 16px; border-radius: 4px; margin: 8px">
Question: Can you find the diet and sample timepoint for the chosen fish sample in the BioSamples record?
</div>

MGnify produces "Analysis summaries" for each Study. These summarise the taxonomic annotations for all the sequencing runs (or assemblies) in the study. They are useful for getting a quick overview of the count of taxa present in each analysed sample.

**Task 4:** For this study study [MGYS00006041](https://www.ebi.ac.uk/metagenomics/studies/MGYS00005851#analysis), we want to find the most common Bacterial phyla in the study using the SSU taxonomic summary.

You could do this by downloading a TSV file and analysing it using something like Excel or R.

Or, you can use the following Python code for a programmatic approach.

**Import packages** : Execute the code below to import the Python libraries required to run the examples presented in this notebook.

In [30]:
# Fetch data via the MGnify API
import requests
from io import StringIO

# Dataframes and display
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Warning verbosity
import warnings 
warnings.filterwarnings(action="ignore")

**Fetch data via the MGnify [API](https://www.ebi.ac.uk/metagenomics/api/v1/)**: fetch the taxonomic study summary at Phylum level for the study and summarise the most prevelant phyla in a table. 

In [31]:
# Define your study ID, pipeline version, and file name
study_id = "MGYS00006041"
pipeline_version = "5.0"
filename = "ERP125469_phylum_taxonomy_abundances_SSU_v5.0.tsv"

# Construct the full URL (notice the base URL is needed!)
url = f"https://www.ebi.ac.uk/metagenomics/api/v1/studies/{study_id}/pipelines/{pipeline_version}/file/{filename}"

# Fetch the file directly
response = requests.get(url)

# Raise an error if request failed
response.raise_for_status()

# Read the text into a pandas dataframe
taxonomies = pd.read_csv(StringIO(response.text), sep='\t')

In [32]:
# Sum the taxonomy counts across all runs in the study
taxonomies['study_total'] = taxonomies.filter(like="ERR").sum(axis=1)

# Limit to phyla in the bacteria superkingdom
bacteria = taxonomies[taxonomies.superkingdom == 'Bacteria']
bacteria.set_index('phylum', inplace=True)

# Show just the most prevalent phyla
bacteria.sort_values(by="study_total", ascending=False).head()

,superkingdom,kingdom,ERZ2627283_FASTA,ERZ2627282_FASTA,ERZ2627276_FASTA,ERZ2627272_FASTA,ERZ2627266_FASTA,ERZ2627265_FASTA,ERZ2627256_FASTA,ERZ2627251_FASTA,ERZ2627250_FASTA,ERZ2627248_FASTA,ERZ2627245_FASTA,ERZ2627244_FASTA,ERR4918740_MERGED_FASTQ,ERR4918718_MERGED_FASTQ,ERR4918701_MERGED_FASTQ,ERR4918697_MERGED_FASTQ,ERR4918693_MERGED_FASTQ,ERR4918674_MERGED_FASTQ,ERR4918667_MERGED_FASTQ,ERR4918665_MERGED_FASTQ,ERR4918660_MERGED_FASTQ,ERR4918656_MERGED_FASTQ,ERR4918653_MERGED_FASTQ,ERR4918609_MERGED_FASTQ,ERR4918597_MERGED_FASTQ,ERR4918528_MERGED_FASTQ,ERR4918521_MERGED_FASTQ,ERR4918503_MERGED_FASTQ,ERR4918491_MERGED_FASTQ,ERR4918438_MERGED_FASTQ,ERR4918435_MERGED_FASTQ,ERR4918431_MERGED_FASTQ,ERR4918420_MERGED_FASTQ,ERR4918408_MERGED_FASTQ,ERR4918397_MERGED_FASTQ,ERR4918412_MERGED_FASTQ,ERR4918751_MERGED_FASTQ,ERR4918729_MERGED_FASTQ,ERR4918716_MERGED_FASTQ,ERR4918702_MERGED_FASTQ,ERR4918696_MERGED_FASTQ,ERR4918691_MERGED_FASTQ,ERR4918685_MERGED_FASTQ,ERR4918684_MERGED_FASTQ,ERR4918671_MERGED_FASTQ,ERR4918659_MERGED_FASTQ,ERR4918745_MERGED_FASTQ,ERR4918735_MERGED_FASTQ,ERR4918728_MERGED_FASTQ,ERR4918695_MERGED_FASTQ,ERR4918688_MERGED_FASTQ,ERR4918683_MERGED_FASTQ,ERR4918657_MERGED_FASTQ,ERR4918655_MERGED_FASTQ,ERR4918654_MERGED_FASTQ,ERR4918652_MERGED_FASTQ,ERR4918651_MERGED_FASTQ,ERR4918650_MERGED_FASTQ,ERR4918649_MERGED_FASTQ,ERR4918648_MERGED_FASTQ,ERR4918647_MERGED_FASTQ,ERR4918646_MERGED_FASTQ,ERR4918645_MERGED_FASTQ,ERR4918644_MERGED_FASTQ,ERR4918643_MERGED_FASTQ,ERR4918642_MERGED_FASTQ,ERR4918640_MERGED_FASTQ,ERR4918639_MERGED_FASTQ,ERR4918638_MERGED_FASTQ,ERR4918637_MERGED_FASTQ,ERR4918636_MERGED_FASTQ,ERR4918635_MERGED_FASTQ,ERR4918634_MERGED_FASTQ,ERR4918633_MERGED_FASTQ,ERR4918632_MERGED_FASTQ,ERR4918631_MERGED_FASTQ,ERR4918630_MERGED_FASTQ,ERR4918628_MERGED_FASTQ,ERR4918627_MERGED_FASTQ,ERR4918626_MERGED_FASTQ,ERR4918623_MERGED_FASTQ,ERR4918621_MERGED_FASTQ,ERR4918619_MERGED_FASTQ,ERR4918614_MERGED_FASTQ,ERR4918547_MERGED_FASTQ,ERR4918433_MERGED_FASTQ,ERR4918426_MERGED_FASTQ,ERR4918394_MERGED_FASTQ,ERR4918641_MERGED_FASTQ,ERR4918719_MERGED_FASTQ,ERR4918722_MERGED_FASTQ,ERR4918698_MERGED_FASTQ,ERR4918666_MERGED_FASTQ,ERR4918663_MERGED_FASTQ,ERR4918625_MERGED_FASTQ,ERR4918540_MERGED_FASTQ,ERR4918530_MERGED_FASTQ,ERR4918526_MERGED_FASTQ,ERR4918523_MERGED_FASTQ,ERR4918516_MERGED_FASTQ,ERR4918430_MERGED_FASTQ,ERR4918428_MERGED_FASTQ,ERR4918404_MERGED_FASTQ,ERR4918403_MERGED_FASTQ,ERR4918399_MERGED_FASTQ,ERR4918748_MERGED_FASTQ,ERR4918743_MERGED_FASTQ,ERR4918737_MERGED_FASTQ,ERR4918733_MERGED_FASTQ,ERR4918723_MERGED_FASTQ,ERR4918712_MERGED_FASTQ,ERR4918709_MERGED_FASTQ,ERR4918705_MERGED_FASTQ,ERR4918704_MERGED_FASTQ,ERR4918700_MERGED_FASTQ,ERR4918699_MERGED_FASTQ,ERR4918694_MERGED_FASTQ,ERR4918690_MERGED_FASTQ,ERR4918689_MERGED_FASTQ,ERR4918687_MERGED_FASTQ,ERR4918686_MERGED_FASTQ,ERR4918682_MERGED_FASTQ,ERR4918681_MERGED_FASTQ,ERR4918680_MERGED_FASTQ,ERR4918679_MERGED_FASTQ,ERR4918678_MERGED_FASTQ,ERR4918677_MERGED_FASTQ,ERR4918676_MERGED_FASTQ,ERR4918673_MERGED_FASTQ,ERR4918672_MERGED_FASTQ,ERR4918670_MERGED_FASTQ,ERR4918669_MERGED_FASTQ,ERR4918668_MERGED_FASTQ,ERR4918664_MERGED_FASTQ,ERR4918662_MERGED_FASTQ,ERR4918661_MERGED_FASTQ,ERR4918658_MERGED_FASTQ,ERR4918629_MERGED_FASTQ,ERR4918624_MERGED_FASTQ,ERR4918620_MERGED_FASTQ,ERR4918617_MERGED_FASTQ,ERR4918553_MERGED_FASTQ,ERR4918551_MERGED_FASTQ,ERR4918548_MERGED_FASTQ,ERR4918542_MERGED_FASTQ,ERR4918538_MERGED_FASTQ,ERR4918534_MERGED_FASTQ,ERR4918437_MERGED_FASTQ,ERR4918436_MERGED_FASTQ,ERR4918434_MERGED_FASTQ,ERR4918432_MERGED_FASTQ,ERR4918429_MERGED_FASTQ,ERR4918427_MERGED_FASTQ,ERR4918425_MERGED_FASTQ,ERR4918424_MERGED_FASTQ,ERR4918423_MERGED_FASTQ,ERR4918422_MERGED_FASTQ,ERR4918421_MERGED_FASTQ,ERR4918419_MERGED_FASTQ,ERR4918418_MERGED_FASTQ,ERR4918417_MERGED_FASTQ,ERR4918416_MERGED_FASTQ,ERR4918415_MERGED_FASTQ,ERR4918414_MERGED_FASTQ,ERR4918413_MERGED_FASTQ,ERR4918411_MERGED_FASTQ,ERR4918410_MERGED_FASTQ,ERR4918409_MERGED_FASTQ,ERR4918407_MERGED_FASTQ,ERR491

During our first search you may have noticed that we selected the study for "Trial A and B". There was another study [MGYS00006086](https://www.ebi.ac.uk/metagenomics/studies/MGYS00006086#overview) for Trial C. Trial C fish samples have a different diet and age. 

**Task 5**: Lets also summarise the top phyla for this Trial C study. 
The endpoint is as follows: "https://www.ebi.ac.uk/metagenomics/api/v1/studies/MGYS00006086/pipelines/5.0/file/ERP136460_phylum_taxonomy_abundances_SSU_v5.0.tsv" Complete the first three empty variables in the code below to get the summary table. 

In [33]:
# Define your study ID, pipeline version, and file name
study_id = ""
pipeline_version = ""
filename = ""

# Construct the full URL (notice the base URL is needed!)
url = f"https://www.ebi.ac.uk/metagenomics/api/v1/studies/{study_id}/pipelines/{pipeline_version}/file/{filename}"

# Fetch the file directly
response = requests.get(url)

# Raise an error if request failed
response.raise_for_status()

# Read the text into a pandas dataframe
taxonomies_c = pd.read_csv(StringIO(response.text), sep='\t')

# Sum the taxonomy counts across all runs in the study
taxonomies_c['study_total'] = taxonomies_c.filter(like="ERR").sum(axis=1)

# Limit to phyla in the bacteria superkingdom
bacteria = taxonomies_c[taxonomies_c.superkingdom == 'Bacteria']
bacteria.set_index('phylum', inplace=True)

# Show just the most prevalent phyla
bacteria.sort_values(by="study_total", ascending=False).head()

HTTPError: 404 Client Error: Not Found for url: https://www.ebi.ac.uk/metagenomics/api/v1/studies//pipelines//file/

<div style="background: #e8d1e8; padding: 16px; border-radius: 4px; margin: 8px">
Question: Is there a difference in top phyla for the different diets. Can we use these results as a confirmation of taxonomic difference?
</div>

Tenericutes is a prevelant phylum for the Trial A & B samples. Lets find all analyses which contain this taxonomic lineage. 

**Task 7**: Return to the "Text Search" feature and this time search for Sample analyses:
* that contain organisms in the Tenericutes lineage 
* also collected from the fish digestive system biome
* analysed with MGnify's pipeline version 5.0 (the latest).

**Task 8**: Select analysis MGYA00607951. Navigate to the **Taxonomic analysis** tab. Taxonomic annotation in this tab is assigned to predicted marker genes in the assembly such as SSU and LSU.

<div style="background: #e8d1e8; padding: 16px; border-radius: 4px; margin: 8px">
Question: According to the charts in the "Taxonomic analysis" section's "SSU" results, what fraction of the phylum composition is assigned to Tenericutes?
</div>

**Task 9**: Now turn to the functional analysis information, in particular **the Pathways/Systems section.** Functional annotations of a metagenomic assembly are based on comparison of proteins coding sequences found in the assembly's contigs to databases of protein function such as KEGG orthologs. KEGG modules are pathways of KEGG orthologs, and a pathway's completeness is determined by the presence or absence of the minimum number of orthologs present to complete that pathway.

<div style="background: #e8d1e8; padding: 16px; border-radius: 4px; margin: 8px">
Question: What is the first 'complete' KEGG Module for this assembly analysis? Complete means all of the Kegg Orthologs in that module are present in the assembly.
</div>

<div style="background: #e8d1e8; padding: 16px; border-radius: 4px; margin: 8px">
Question: Does this mean that the Kegg Module you just found is a function performed by organisms within the taxa we originally searched for (Tenericutes)?
</div>

<div style="background: #d0debb; padding: 16px; border-radius: 4px; margin: 8px">
Fun extras! You can search analyses again. This time for "Amplicon" data with Tenericutes from the fish digestive system. What is different about these taxonomic annotations vs assembly? 
</div>

---

## Section 2: MGnify Genomes

To find functional annotations for a specific species, we can use metagenome assembly genomes (MAGs) – draft genomes made by taxonomically binning assembled contigs.
[MGnify Genomes](https://www.ebi.ac.uk/metagenomics/browse/genomes) is MGnify's resource for MAGs, where MAGs derived from various biomes are organised into species-level clusters and annotated with a suite of functional annotation tools.

Let's assume we are particularly interested in the taxa Tenericutes - the most prevalent Bacterial phylum in the study from Part 1. As is a symptom of fast moving scientific fields, Tenericutes is designated a different phylum by [GTDB](https://gtdb.ecogenomic.org/) - the taxonomic classification system used for MGnify genomes!

The equivalent lineage is Bacteria > Firmicutes > Bacilli

**Task 10**: Navigate to the genomes resource using the **Browse Data** option at the top of the page. You will see a list of biome specific genome catalogues. We are most likely to find similar species to our assembly data in the same biome. In this case the **non-model-fish-gut-v2-0** catalogue. Click on this catalogue.

<div style="background: #e8d1e8; padding: 16px; border-radius: 4px; margin: 8px">
Question: In the taxonomy tree find our lineage of interest. How many genomes are in this lineage?
</div>

**Task 11**: Lets have a look at any genome of interest here. For example Lactobacillus johnsonii. Navigate to this genome via the taxonomic tree:
Lactobacillales > Lactobacillaceae > Lactobacillus > Lactobacillus johnsonii > MGYG000299657

<div style="background: #e8d1e8; padding: 16px; border-radius: 4px; margin: 8px">
Question: What is the completeness and contamination of this genome?
</div>

<div style="background: #e8d1e8; padding: 16px; border-radius: 4px; margin: 8px">
Question: What is the most prevelant KEGG modules for this genome?
</div>

**Task 12**: Looking at all the data you have available for this MAG, and exploring the other features available in the MGnify Genomes resource, can you work out how you might search for genomes similar to this one in other catalogues?

<div style="background: #bbd2de; padding: 16px; border-radius: 4px; margin: 8px">
Hint: Navigate back to the main genomes page and look at the other options available. 
</div>